# Building and Training Model on the cleaned data file

## 1. Istalling and importing Packages

In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
## using smotenning method to do balance the label data to remove the accuracy curse 
## we can do this by upsampling or downsampling too.
!pip install imblearn
from imblearn.combine import SMOTEENN

In [6]:
modelData=pd.read_csv("C:/Users/Nexgen/Telecom data/cleanedData.csv")
modelData.head()

,SeniorCitizen,MonthlyCharges,TotalCharges,Churn,gender_Female,gender_Male,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,...,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,tenure_group_1 - 12,tenure_group_13 - 24,tenure_group_25 - 36,tenure_group_37 - 48,tenure_group_49 - 60,tenure_group_61 - 72
0,0,29.85,29.85,0,1,0,0,1,1,0,...,0,0,1,0,1,0,0,0,0,0
1,0,56.95,1889.50,0,0,1,1,0,1,0,...,0,0,0,1,0,0,1,0,0,0
2,0,53.85,108.15,1,0,1,1,0,1,0,...,0,0,0,1,1,0,0,0,0,0
3,0,42.30,1840.75,0,0,1,1,0,1,0,...,1,0,0,0,0,0,0,1,0,0
4,0,70.70,151.65,1,1,0,1,0,1,0,...,0,0,1,0,1,0,0,0,0,0


## 2. Separating features and label from cleaned data file

In [119]:
featuresData=df.drop('Churn',axis=1)
featuresData
#storing separate features file into a separate csv
featuresData.to_csv('featuressData.csv', index=False)

In [120]:
labelData=df['Churn']
labelData
#storing separate label file into a separate csv
labelData.to_csv('labelData.csv', index=False)

## 3. creating and training Decision Tree Model

In [10]:
x_train,x_test,y_train,y_test=train_test_split(featuresData,labelData,test_size=0.2)

In [12]:
#Decision tree classifier
model_dt=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [13]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [14]:
model_dt.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [15]:
y_pred=model_dt.predict(x_test)
y_pred

array([0, 0, 0, ..., 0, 1, 1], dtype=int64)

In [16]:
model_dt.score(x_test,y_test)

0.7938877043354655

Here accuracy is very low 
### possible reason was identified by us during EDA that our label data is highly imbalanced
classification report and confusion matrix below wil give us the clear image of our model prediction accuracy

In [17]:
print(classification_report(y_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       999
           1       0.67      0.57      0.62       408

    accuracy                           0.79      1407
   macro avg       0.75      0.73      0.74      1407
weighted avg       0.79      0.79      0.79      1407



In [18]:
print(confusion_matrix(y_test,y_pred))

[[883 116]
 [174 234]]


as we can see that precision, recall and f1-score for churners is very low 
so we need to solve this data imbalanced issue before moving on to trianing and predicting results

## 4. smoteenning the dataset to get balanced ratio for label data

In [25]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(featuresData,labelData)

In [26]:

xr_train,xr_test,yr_train,yr_test=train_test_split(X_resampled, y_resampled,test_size=0.2)

## 5.Decision Tree Classifier after smoteenning

In [52]:
model_dt_smote=DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8)

In [60]:
model_dt_smote.fit(xr_train,yr_train)
yr_predict = model_dt_smote.predict(xr_test)
model_score_r = model_dt_smote.score(xr_test, yr_test)
print(model_score_r)
print(metrics.classification_report(yr_test, yr_predict))

0.9317988064791134
              precision    recall  f1-score   support

           0       0.95      0.90      0.92       537
           1       0.92      0.96      0.94       636

    accuracy                           0.93      1173
   macro avg       0.93      0.93      0.93      1173
weighted avg       0.93      0.93      0.93      1173



In [59]:
print(metrics.confusion_matrix(yr_test, yr_predict))

[[481  56]
 [ 24 612]]


In [55]:
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(yr_test, yr_predict)
print("MAE: ", mae)

# calculate the coefficient of determination (R-squared)
r2 = r2_score(yr_test, yr_predict)
print("R-squared: ", r2)

MAE:  0.06820119352088662
R-squared:  0.7252380450440954


This is a good result, as the MAE is low and suggests that our model is making accurate predictions.
as the R-squared is reasonably high and suggests that our model is a good fit for the data. Lets see if r-squared value can be improved by using random forest classifier

# 6.Random Forest Classifier

In [32]:
from sklearn.ensemble import RandomForestClassifier

In [97]:
model_rf=RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8)

In [98]:
model_rf.fit(xr_train,yr_train)

RandomForestClassifier(max_depth=6, min_samples_leaf=8, random_state=100)

In [99]:
y_pred=model_rf.predict(xr_test)

In [100]:
model_rf.score(xr_test,yr_test)

0.9343563512361467

In [101]:
print(classification_report(yr_test, y_pred, labels=[0,1]))

              precision    recall  f1-score   support

           0       0.97      0.88      0.93       537
           1       0.91      0.98      0.94       636

    accuracy                           0.93      1173
   macro avg       0.94      0.93      0.93      1173
weighted avg       0.94      0.93      0.93      1173



In [102]:
print(metrics.confusion_matrix(yr_test, y_pred))

[[475  62]
 [ 15 621]]


In [103]:
from sklearn.metrics import mean_absolute_error, r2_score

mae = mean_absolute_error(yr_test, y_pred)
print("MAE: ", mae)

# calculate the coefficient of determination (R-squared)
r2 = r2_score(yr_test, y_pred)
print("R-squared: ", r2)

MAE:  0.06564364876385337
R-squared:  0.7355416183549418


# 7.Further Suggested improvements in this model
this model can be integrated inside a pipeline using sklearn pipiline module 
for classifier sleection (for selecting the most accurate classifier)
hyper paramter tuning (for getting highest possible accuracy)
later cross validation can be done using cross_validate module of sklearn.
Here in my case i have acheived a reasonably good accuracy for the prediction whether the custome is going to churn or not. 

# 8.Saving the trained Models

In [104]:
import pickle

In [114]:
filename = 'decsionTree.sav'
filename1 = 'randomeForest.sav'

In [117]:
pickle.dump(model_dt_smote, open(filename, 'wb'))


In [116]:
pickle.dump(model_rf, open(filename1, 'wb'))